<a href="https://colab.research.google.com/github/Mueezuddin56/Mueezuddin56/blob/main/computer%20vision%20project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/data/image"

origin.7z.001  origin.7z.003  origin.7z.005  origin.7z.007
origin.7z.002  origin.7z.004  origin.7z.006  origin.7z.008


In [ ]:
!ls
!7za -y x "/content/drive/MyDrive/Colab Notebooks/data/image/origin.7z.*"

drive  sample_data

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Colab Notebooks/data/image/                                                            8 files, 8113576419 bytes (7738 MiB)
  0%    
Extracting archive: /content/drive/MyDrive/Colab Notebooks/data/image/origin.7z.001
--
Path = /content/drive/MyDrive/Colab Notebooks/data/image/origin.7z.001
Type = Split
Physical Size = 1048576000
Volumes = 8
Total Physical Size = 8113576419
----
Path = origin.7z
Size = 8113576419
--
Path = origin.7z
Type = 7z
Physical Size = 8113576419
Headers Size = 863607
Method = LZMA:25
Solid = +
Blocks = 2

  0%      0% - origin/afraid_African_214.jp

In [5]:
!mkdir "/content/drive/MyDrive/Colab Notebooks/new_data"
!cp -r ./origin "/content/drive/MyDrive/Colab Notebooks/new_data/"


mkdir: cannot create directory ‘/content/drive/MyDrive/Colab Notebooks/new_data’: File exists
cp: cannot stat './origin': No such file or directory


In [6]:
import numpy as np
import os
import glob
import pandas as pd
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [7]:
images_dir = "/content/drive/MyDrive/Colab Notebooks/new_data/origin"

In [8]:
images_dir = "/content/drive/MyDrive/Colab Notebooks/new_data/origin"
image_count = len(list(glob.glob(os.path.join(images_dir, '*.jpg'))))
print("Total images:", image_count)

Total images: 22186


In [9]:
images = list(glob.glob(os.path.join(images_dir, '*.jpg')))
for image_path in images[:10]:
    img =Image.open(image_path)
    img.show()

In [24]:
labels_file = "/content/drive/MyDrive/Colab Notebooks/data/label/label.lst"
import pandas as pd
df_info = pd.read_csv(labels_file, sep=' ', header=None)
col_names = "image_name face_id_in_image face_box_top face_box_left face_box_right face_box_bottom face_box_cofidence expression_label".split(' ')
df_info.columns = col_names
df_info


,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [25]:
for ind in df_info.index:
  df_iterate = df_info['image_name'][ind], df_info['face_box_cofidence'][ind]

print(df_iterate)

('surprised_expression_394.jpg', 77.7758)


In [26]:
labels = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [27]:
df_sel = df_info[df_info.face_box_cofidence > 65]
df_sel

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0
5,angry_actor_137.jpg,0,93,468,842,467,88.9519,0
9,angry_actor_150.jpg,0,56,263,376,169,81.8792,0
11,angry_actor_16.jpg,0,158,249,521,430,66.4535,0
...,...,...,...,...,...,...,...,...
91783,afraid_woman_204.jpg,0,20,13,93,100,74.5950,6
91787,expressionless_husband_673.jpg,0,48,194,388,242,82.2975,4
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5


In [29]:
for i, row in df_sel.iterrows():
  image_name = row['image_name']
  x1 = row['face_box_left']
  x2 = row['face_box_right']
  y1 = row['face_box_top']
  y2 = row['face_box_bottom']
  a = [x1,x2, y1,y2]

print(row)

image_name            surprised_expression_394.jpg
face_id_in_image                                 0
face_box_top                                    47
face_box_left                                   38
face_box_right                                 152
face_box_bottom                                161
face_box_cofidence                         77.7758
expression_label                                 5
Name: 91792, dtype: object


In [37]:
import os
import cv2
from tqdm import tqdm_gui

X = []
Y = []

images_dir= "/content/drive/MyDrive/Colab Notebooks/new_data/origin"
cropped_directory ="/content/drive/MyDrive/Colab Notebooks/new_data/cropped"
for i, row in df_sel.sample(1000).iterrows():
    image_name = row['image_name']
    x1 = row['face_box_left']
    x2 = row['face_box_right']
    y1 = row['face_box_top']
    y2 = row['face_box_bottom']
    label = row['expression_label']

    cropped_path = os.path.join(cropped_directory, f"cropped_{image_name}")

    # Read the cropped image
    comb_path = os.path.join(images_dir, image_name)
    cropped_image = cv2.imread(comb_path)

    if cropped_image is not None:
        face = cropped_image[x1:x2, y1:y2]

        # Debugging: Print face shape and cropping coordinates
        print(f"Face shape: {face.shape}, x1={x1}, x2={x2}, y1={y1}, y2={y2}")
    else:
        continue

    if face is not None and not face.size == 0:
        face_resized = cv2.resize(face, (64, 64))
    else:
        continue

    X.append(face_resized)
    Y.append(label)

Face shape: (328, 328, 3), x1=109, x2=437, y1=109, y2=437
Face shape: (112, 112, 3), x1=75, x2=187, y1=37, y2=149
Face shape: (383, 383, 3), x1=831, x2=1214, y1=191, y2=574
Face shape: (194, 194, 3), x1=32, x2=226, y1=32, y2=226
Face shape: (56, 56, 3), x1=94, x2=150, y1=37, y2=93
Face shape: (159, 159, 3), x1=213, x2=372, y1=106, y2=265
Face shape: (157, 157, 3), x1=52, x2=209, y1=52, y2=209
Face shape: (125, 133, 3), x1=289, x2=422, y1=55, y2=188
Face shape: (132, 226, 3), x1=1324, x2=1550, y1=245, y2=471
Face shape: (115, 115, 3), x1=57, x2=172, y1=28, y2=143
Face shape: (134, 228, 3), x1=666, x2=894, y1=266, y2=494
Face shape: (272, 273, 3), x1=683, x2=956, y1=136, y2=409
Face shape: (80, 80, 3), x1=188, x2=268, y1=121, y2=201
Face shape: (115, 115, 3), x1=326, x2=441, y1=239, y2=354
Face shape: (197, 197, 3), x1=32, x2=229, y1=49, y2=246
Face shape: (67, 67, 3), x1=158, x2=225, y1=107, y2=174
Face shape: (0, 135, 3), x1=849, x2=984, y1=33, y2=168
Face shape: (0, 96, 3), x1=459, x2

In [38]:
batch_size = 32
img_height = 180
img_width = 180

In [39]:
# Function to load and preprocess an image
def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img

In [40]:
print(len(X))
print(len(Y))

297
297


In [42]:
X = np.array(X)
Y = np.array(Y)

In [43]:
print(len(X))


297


In [44]:
if isinstance(X, np.ndarray) and isinstance(Y, np.ndarray):
    print("Images are converted to NumPy arrays.")
else:
    print("Images are not converted to NumPy arrays.")


Images are converted to NumPy arrays.


In [45]:
#X = X/255.0
X_normalized = X.astype('float32') / 255.0

# Check the shape of the array
print("Shape of X_normalized:", X_normalized.shape)

Shape of X_normalized: (297, 64, 64, 3)


In [46]:
if np.min(X_normalized) >= 0.0 and np.max(X_normalized) <= 1.0:
    print("Images are properly normalized.")
else:
    print("Images are not properly normalized. Pixel value range: [", np.min(X_normalized), ",", np.max(X_normalized), "]")

Images are properly normalized.


In [47]:
#reshaping images for oversamlping
X_new = X.reshape(-1, 64*64*3)

In [53]:
# oversampling the images
from imblearn.over_sampling import SMOTE
k_neighbors = 2  # Choose the number of nearest neighbors for SMOTE
sampler = SMOTE(k_neighbors=k_neighbors)

# Apply SMOTE to oversample the data
XSMOTE, YSMOTE = sampler.fit_resample(X_new, Y)

In [54]:
pd.Series(YSMOTE).value_counts()

1    147
5    147
3    147
6    147
0    147
4    147
2    147
dtype: int64

In [55]:
# again bringing images to its original shape
X = XSMOTE.reshape(-1, 64, 64, 3)

In [56]:
#  label encoding, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded.reshape(-1, 1))

# Check the shape of onehot_encoded
print("Shape of onehot_encoded:", onehot_encoded.shape)

Shape of onehot_encoded: (297, 7)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [57]:
# Check the first few rows of onehot_encoded
print("First few rows of onehot_encoded:")
print(onehot_encoded[:5])

First few rows of onehot_encoded:
[[0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]


In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True
)

datagen.fit(X)

In [65]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing
# Assuming X_normalized and onehot_encoded are appropriately prepared

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, onehot_encoded, test_size=0.2, random_state=42)

# Data Augmentation (optional)
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(onehot_encoded.shape[1], activation='softmax'))  # Use the number of classes from onehot_encoded

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 50

# Without data augmentation
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# With data augmentation (uncomment the following lines)
# model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
#           steps_per_epoch=len(X_train) // batch_size, epochs=epochs, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")


Epoch 1/50
7/7 [==============================] - 1s 42ms/step - loss: 7.2411 - accuracy: 0.2629 - val_loss: 1.9395 - val_accuracy: 0.0000e+00
Epoch 2/50
7/7 [==============================] - 0s 13ms/step - loss: 5.7541 - accuracy: 0.3850 - val_loss: 1.6531 - val_accuracy: 0.3333
Epoch 3/50
7/7 [==============================] - 0s 13ms/step - loss: 4.0635 - accuracy: 0.4930 - val_loss: 1.8002 - val_accuracy: 0.3333
Epoch 4/50
7/7 [==============================] - 0s 12ms/step - loss: 3.5516 - accuracy: 0.5540 - val_loss: 1.6766 - val_accuracy: 0.4167
Epoch 5/50
7/7 [==============================] - 0s 13ms/step - loss: 2.4636 - accuracy: 0.5775 - val_loss: 1.9973 - val_accuracy: 0.4583
Epoch 6/50
7/7 [==============================] - 0s 18ms/step - loss: 1.8789 - accuracy: 0.5915 - val_loss: 2.1792 - val_accuracy: 0.4583
Epoch 7/50
7/7 [==============================] - 0s 20ms/step - loss: 1.2465 - accuracy: 0.6714 - val_loss: 2.4499 - val_accuracy: 0.4583
Epoch 8/50
7/7 [=======